In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch import tensor
from torch.autograd import Variable
from generate_data import generate_data, data_loader

In [66]:
batch_size = 32

lr = 0.00005

loss_fn = nn.CrossEntropyLoss()

In [67]:
N = 4*int(1e4)

X, y = generate_data(N, is_short=True, type = 2)

X_test, y_test = generate_data(2000, is_short=False, type = 2)

In [71]:
class RNN_predictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNN_predictor, self).__init__()
        
        self.hidden_dim = hidden_dim
        
        self.layer_dim = layer_dim
        
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, nonlinearity='relu')
        
        self.fc = nn.Linear(hidden_dim, output_dim)
    
        self.activation = nn.Softmax()
        
    def forward(self, x):
        
        # Initialize hidden state with zeros
        # hidden = Variable(torch.zeros(self.layer_dim, x.shape[1], self.hidden_dim))

        # One time step
        out, hn = self.rnn(x)
        out = self.fc(out[ -1,:, :])
        
        return self.activation(out)


model = RNN_predictor(3, 10, 1, 2)

In [72]:
print(model)

RNN_predictor(
  (rnn): RNN(3, 10)
  (fc): Linear(in_features=10, out_features=2, bias=True)
  (activation): Softmax(dim=None)
)


In [73]:
def test(model):
    model.eval()
    test_batches = data_loader(X_test,y_test,batch_size=32, min_length = 50)
    acc = 0.0
    i = 0
    for data, labels in test_batches:
        pred = model(data).view((-1,2))
        corr = (torch.argmax(pred,1) == torch.argmax(labels,1)).float().mean().item()
        acc += corr
        i += 1
    return acc/i

def train(model,n_epochs):
    parameters = model.parameters()

    optimizer = torch.optim.Adam(parameters, lr=lr)

    model.train()


    for i in range(n_epochs):
        model.train()
        train_batches = data_loader(X,y,batch_size=32, min_length = 20)

        sum_loss = 0.0
        i_loss = 0
        for data, labels in train_batches:
            pred = model(data)
            loss_fn = F.binary_cross_entropy
            loss = loss_fn(pred,labels)
            loss.backward()
            optimizer.step()

            sum_loss += loss.item()
            i_loss += 1
        print(f"loss {sum_loss/i_loss:.3} - acc {test(model):.3}")


train(model,80)

c:\Users\jakob\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


loss 0.69 - acc 0.532
loss 0.655 - acc 0.781
loss 0.525 - acc 0.793
loss 0.405 - acc 0.816
loss 0.337 - acc 0.836
loss 0.383 - acc 0.782
loss 0.382 - acc 0.796
loss 0.328 - acc 0.83
loss 0.282 - acc 0.908
loss 0.28 - acc 0.819
loss 0.436 - acc 0.775
loss 0.548 - acc 0.539
loss 0.622 - acc 0.504
loss 0.644 - acc 0.542
loss 0.648 - acc 0.502
loss 0.648 - acc 0.501
loss 0.643 - acc 0.56
loss 0.636 - acc 0.526
loss 0.628 - acc 0.555
loss 0.619 - acc 0.563
loss 0.595 - acc 0.663
loss 0.498 - acc 0.769
loss 0.399 - acc 0.792
loss 0.399 - acc 0.8
loss 0.384 - acc 0.796
loss 0.396 - acc 0.786
loss 0.398 - acc 0.792
loss 0.397 - acc 0.789
loss 0.395 - acc 0.782
loss 0.392 - acc 0.785
loss 0.385 - acc 0.784


KeyboardInterrupt: 